# Exploring a Data Repository

<br>Owner: **Phil Marshall** ([@drphilmarshall](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@drphilmarshall))
<br>Last Verified to Run: **2018-09-07**
<br>Verified Stack Release: **16.0**

This notebook shows how to find out what's in a data repository, and how to find out which inputs went into each component of it.  

### Learning Objectives:
After working through and studying this notebook you should be able to understand how to use the Butler to figure out: 
   1. Which data types are present in a data repository
   2. If coadds have been made, what the available tracts and patches are
   3. TBD
   
### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.


## Set Up

In [ ]:
import os
import sys
import warnings
import matplotlib.pyplot as plt
%matplotlib inline

# Filter some warnings printed by v16.0 of the stack
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

## The HSC Dataset: What's in there?
We'll use the `hsc` dataset as our testing ground, and start by figuring out what's there.

We'll need a butler to interrogate the `hsc` data repository.

In [ ]:
from lsst.daf.persistence import Butler

# Instantiate the butler to bring us some HSC data.

depth = 'WIDE' # WIDE, DEEP, UDEEP
field = 'SSP_WIDE' # SSP_WIDE, SSP_DEEP, SSP_UDEEP

repo = '/datasets/hsc/repo/rerun/DM-13666/%s/'%(depth)
butler = Butler(repo)

print(repo)

In [ ]:
# datasetRefOrType : forced_src
# see all options at
# /opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/obs_subaru/16.0+1/python/lsst/obs/hsc

# Better to point to a resource online:

In [ ]:
from stackclub import where_is

In [ ]:
# where_is('ingest.py', in_the='source', assuming_its_a='cmdlinetask')

The `HscMapper` class is defined in [HscMapper.py](https://github.com/lsst/obs_subaru/blob/master/python/lsst/obs/hsc/hscMapper.py). Let's read about it.

In [ ]:
from lsst.obs.hsc import HscMapper

In [ ]:
# help(HscMapper)

The mapper defines a (large) number of different dataset types. Some of these are specific to this particular dataset, others are more general. Even filtering out some intermediate dataset types, we are still left with a long list...

In [ ]:
mapper = HscMapper(root=repo)
all_dataset_types = mapper.getDatasetTypes()

remove = ['_config', '_filename', '_md', '_sub', '_len', '_schema', '_metadata']

shortlist = []
for dataset_type in all_dataset_types:
    keep = True
    for word in remove:
        if word in dataset_type:
            keep = False
    if keep:
        shortlist.append(dataset_type)

print(shortlist)

The `butler` purpoorts to be able to check whether a dataset actually exists or not, but needs a specific dataset ID to check it:

In [ ]:
butler.datasetExists('calexp', dataId={})

Instead, one can try querying the metadata and checking for an error.

In [ ]:
datasettype = 'calexp'

try:
    datasetkeys = butler.getKeys(datasettype)
    onekey = list(datasetkeys.keys())[0]
    metadata = butler.queryMetadata(datasettype, [onekey])
    print("{} dataset exists.".format(datasettype))
except:
    print("{} dataset doesn't exist.".format(datasettype))

In [ ]:
# help(butler)

### Return basic dataset properties
For this dataset, we can look at the filters used, number of visits, number of pointings, etc. by examining the Butler's keys and metadata:

In [ ]:
# Interesting dataset types for the HSC_mapper dataset. 
datasettypes = ['calexp', 'calexpBackground', 'icSrc', 
                'src', 'srcMatch', 'srcMatchFull', 'ossImage', 
                'flattenedImage', 'wcs', 'fcr', 'photoCalib',
                'jointcal_wcs', 'jointcal_photoCalib', 'skyCorr',
                'calexp_camera', 'brightObjectMask', 'deepCoadd_calexp', 
                'deepCoadd_det', 'deepCoadd_meas', 'deepCoadd_measMatch', 
                'deepCoadd_mergeDet', 'deepCoadd_ref', 'deepCoadd_forced_src', 
                'forced_src' ]

For these basic properties, we will look at the `calexp` and `src` tables.

In [ ]:
# This would be faster if only one query were issued
visits = butler.queryMetadata('calexp', ['visit'])
pointings = butler.queryMetadata('calexp', ['pointing'])
ccds = butler.queryMetadata('calexp', ['ccd'])
fields = butler.queryMetadata('calexp', ['field'])
filters = butler.queryMetadata('calexp', ['filter'])

# Collect number of objects from Source Catalog
sources = butler.queryMetadata('src', ['id'])

In [ ]:
num_visits = len(visits)
num_pointings = len(pointings)
num_ccds = len(ccds)
num_fields = len(fields)
num_filters = len(filters)

num_sources = len(sources)

In [ ]:
from IPython.display import display, Markdown
import numpy as np

One key quantity for astronomers is the total sky area imaged. We can estimate this from the coadd tract info.

In [ ]:
# Collect tracts from files
import os, glob
tracts = sorted([int(os.path.basename(x)) for x in
                 glob.glob(os.path.join(repo, 'deepCoadd-results', 'merged', '*'))])
num_tracts = len(tracts)

#Note: I'd like to do this with the butler, but it appears 'tracts' have to be
#      specified in the dataId to be queried, so the queryMetadata method fails

In [ ]:
# Calculate area from all tracts
skyMap = butler.get('deepCoadd_skyMap')
total_area = 0.0  #deg^2
for test_tract in tracts:
    # Get inner vertices for tract
    tractInfo = skyMap[test_tract]
    vertices = tractInfo._vertexCoordList
    
    #calculate area of box
    av_dec = 0.5 * (vertices[2][1] + vertices[0][1])
    av_dec = av_dec.asRadians()
    delta_ra_raw = vertices[0][0] - vertices[1][0] 
    delta_ra = delta_ra_raw.asDegrees() * np.cos(av_dec)
    delta_dec= vertices[2][1] - vertices[0][1]
    area = delta_ra * delta_dec.asDegrees()
    
    #combine areas
    total_area += area
    
print("Total area imaged (sq deg): ",total_area)

#round total area for table purposes
rounded_total_area = round(total_area, 2)
    

In [ ]:
# Print out a report of the metadata

# dataset_name = 'HSC'
# display(Markdown('# Dataset: %s' %dataset_name))

# A more automated version of the table title:
dataset_name = 'HSC'
display(Markdown('# %s' % repo))


# Make a table of the collected metadata
collected_data = [num_visits, num_pointings, num_ccds, num_fields, num_filters, num_sources, 
                  num_tracts, rounded_total_area]
data_names = ("Number of Visits", "Number of Pointings", "Number of CCDs", "Number of Fields", 
              "Number of Filters", "Number of Sources", "Number of Tracts", "Total Sky Area (deg$^2$)")

output_table = "|   Metadata Characteristics  |  | \n  | :---: | --- | \n "
counter = 0
while counter < len(collected_data):
    output_table += "| %s |  %s | \n" %(data_names[counter], collected_data[counter])
    counter += 1
display(Markdown(output_table))

# Show which fields and filters we're talking about:
display(Markdown('Fields: (%i total)' %num_fields))
print(fields)
display(Markdown('Filters: (%i total)' %num_filters))
print(filters)


## Sandbox: Initial Exploration of Tracts and Patches, Coadd Images

_To be deleted eventually..._

Let's try exploring the `hsc` dataset's coadd images, and the visits that went into them. Jim Bosch shows how to do this in [this community.lsst.org post](https://community.lsst.org/t/visualizing-source-images-in-a-coadd/441/2).

We'll need a single coadd image to work from.

In [ ]:
from lsst.daf.persistence import Butler

# Instantiate the butler
depth = 'WIDE' # WIDE, DEEP, UDEEP
field = 'SSP_WIDE' # SSP_WIDE, SSP_DEEP, SSP_UDEEP
repo = '/datasets/hsc/repo/rerun/DM-13666/%s/'%(depth)
butler = Butler(repo)

# The following does not work, because ci_hsc has not been ingested or reduced!
# butler = Butler('/project/shared/data/ci_hsc/')

In [ ]:
# Jim's function for returning the visits that went into a coadd:
def showInputs(butler, dataId, coaddType="deepCoadd"):
    coadd = butler.get(coaddType, **dataId)
    visitInputs = coadd.getInfo().getCoaddInputs().visits
    ccdInputs = coadd.getInfo().getCoaddInputs().ccds
    ccdDict = dict((int(v), int(ccd)) for v, ccd in zip(ccdInputs.get("visit"), ccdInputs.get("ccd")))
    for v in visitInputs.get("id"):
        md = butler.get("calexp_md", visit=int(v), ccd=ccdDict[v])
        print("%d %4.0f" % (v, afwImage.Calib(md).getExptime()))

In [ ]:
# Find all tracts, and choose the first one we come to:
import os, glob
tracts = sorted([int(os.path.basename(x)) for x in
                 glob.glob(os.path.join(repo, 'deepCoadd-results', 'merged', '*'))])
tract = tracts[15]

In [ ]:
# Find all patches in our tract, and choose the first one we come to. For this, we need the skymap:
skyMap = butler.get('deepCoadd_skyMap')
tractInfo = skyMap[tract]
patchInfo = tractInfo.getPatchInfo([0, 0])

In [ ]:
tractInfo

In [ ]:
patchInfo

In [ ]:
# Obtain a random dataid for a coadd image in the i-band
band = 'HSC-I'
patch = (0,0)

subset = butler.subset('deepCoadd', dataId={'filter':band, 'tract':tract, 'patch':patch})
dataid = subset.cache[0]
print(dataid)

In [ ]:
coadd = butler.get('deepCoadd', **dataid)

In [ ]:
showInputs(butler, dataid)